# Import Libraries

In [1]:
from swarmintelligence import GeneticAlgorithm, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']

In [3]:
gray_misc_dataset

{'AirplaneF16.tiff': array([[ 65, 199, 193, ..., 166, 164, 148],
        [ 54, 196, 192, ..., 179, 155, 140],
        [ 54, 197, 194, ..., 190, 159, 141],
        ...,
        [ 67, 213, 213, ..., 184, 176, 151],
        [ 68, 214, 214, ..., 166, 184, 167],
        [ 89,  86,  88, ..., 112, 111, 107]], dtype=uint8),
 'Lena.png': array([[162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        ...,
        [ 43,  43,  50, ..., 104, 100,  98],
        [ 44,  44,  55, ..., 104, 105, 108],
        [ 44,  44,  55, ..., 104, 105, 108]], dtype=uint8),
 'Male.tiff': array([[130, 138, 140, ..., 115, 113, 111],
        [139, 134, 131, ..., 110, 117, 116],
        [141, 127, 131, ..., 109, 108, 107],
        ...,
        [102, 103, 103, ...,  76,  80,  89],
        [106, 102, 109, ...,  72,  69,  82],
        [ 98, 105, 116, ...,  85,  81,  89]], dtype=uint8),
 'Mandrill.tiff': array([[145,  56,  49, ..., 113, 153, 177],

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/GA/optimal_params.pkl', 'rb'))
optimal_params

{'pop_size': 25, 'max_gens': 100, 'mr': 0.2, 'cr': 0.6}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['mMasi_entropy']
objs = ['max']
n_runs = 30

In [6]:
ga_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds':[],
    'Mean CPU_time (seconds)': [],
    'Mean Fitness':[],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions': [],
    'CPU_time (seconds)': [],
    'Fitness':[],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in gray_misc_dataset.items():
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                ga_evaluation_results['image_name'].append(image_name)
                ga_evaluation_results['thresholds'].append(threshold)
                ga_evaluation_results['fitness_function'].append(fitness_function)
                ga_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                fitness_values = []
                list_best_thresholds = []
                list_fitness = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # GA
                    ga = GeneticAlgorithm(
                        k=threshold,
                        pop_size=optimal_params['pop_size'],
                        max_gens=optimal_params['max_gens'],
                        fitness_function=fitness_function,
                        mr=optimal_params['mr'],
                        cr=optimal_params['cr'],
                    )
                    # Evaluation Metrics
                    st = time.time()
                    populations, best_thresholds = ga.fit_run(image_array)
                    et = time.time()
                    regions = Utilization().digitize(image_array, best_thresholds)
                    list_cpu_time.append(et-st)
                    list_best_thresholds.append(best_thresholds)
                    fitness_values.append(ga.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, regions))
                    list_rmse.append(rmse(image_array, regions))
                    list_psnr.append(psnr(image_array, regions))
                    list_ssim.append(ssim(image_array, regions)[0])
                    list_uqi.append(uqi(image_array, regions))
                    print(ga.get_params_training_()['best_fitness_tracking'][-1])
                
                mean_best_thresholds = np.round(np.array(list_best_thresholds).mean(axis=0)).astype('int')
                mean_regions=Utilization().digitize(image_array, mean_best_thresholds)
                ga_evaluation_results['Mean best_thresholds'].append(mean_best_thresholds)
                ga_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                ga_evaluation_results['Mean Fitness'].append(np.mean(np.array(fitness_values)))
                ga_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                ga_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                ga_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                ga_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                ga_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                ga_evaluation_results['Regions'].append(mean_regions)
                ga_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                ga_evaluation_results['Fitness'].append(fitness_values)
                ga_evaluation_results['MSE'].append(list_mse)
                ga_evaluation_results['RMSE'].append(list_rmse)
                ga_evaluation_results['PSNR'].append(list_psnr)
                ga_evaluation_results['SSIM'].append(list_ssim)
                ga_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean(np.array(fitness_values))}")

	 AirplaneF16.tiff | Threshold =  2
1938.398305333441
1921.2857090901718
1946.809716789367
1926.7898533914545
1942.5065759152849
1898.5825554575094
1932.4804629295436
1932.2044981836962
1911.9495918583075
1936.529670612877
1938.354946709479
1942.932167432076
1919.5270898409278
1930.6439740498201
1933.1202681241634
1935.1711960913817
1937.899601835227
1947.1797867035743
1895.0444077874959
1937.9703009765838
1924.0973834159624
1930.0692963755257
1894.0338467596337
1948.0316801449706
1936.9186726990824
1947.0784630468856
1945.9864464927632
1926.6601446935126
1943.5831120505559
1946.785251509361
Mean Fitness = 1931.620832543354
	 AirplaneF16.tiff | Threshold =  3
2012.6139386688455
1999.9556337972517
2009.0897515217243
2023.3318070164974
2003.700404066616
2006.0531327900865
2016.4563224319181
2018.090735840415
2012.0149533012425
1990.7319194778802
2019.8598695773408
1996.0124354894451
2011.7990522011987
1974.518790409672
2001.4895940633346
2010.5609417353303
1987.244589929814
1984.96973090

3207.148854008028
Mean Fitness = 3189.891884296177
	 Mandrill.tiff | Threshold =  2
1432.3185074655353
1525.2813753657015
1535.9794058555178
1508.1825555385956
1537.8591070557677
1542.870554807098
1539.6262793530577
1533.8703748708358
1458.8571714229497
1515.4070927224354
1547.5272315688856
1498.8328779109954
1523.9859878721352
1482.1285520845918
1539.298367371996
1543.7822492548846
1543.2426690590369
1523.9831682743752
1526.4058943918835
1547.2763158577973
1442.3233806726269
1516.4734357573643
1529.2762841535477
1545.32434731896
1510.2930698399305
1452.5145177435693
1526.499041026008
1520.9877282774632
1528.9916834339524
1541.4567165199524
Mean Fitness = 1517.3618647615817
	 Mandrill.tiff | Threshold =  3
1606.3210937068866
1599.1655824332483
1568.600745271468
1618.0689881413389
1622.521765234635
1599.5751069083622
1626.367204158144
1606.4088028116698
1610.4085518066179
1624.739727715169
1612.0254408996366
1554.8111594353486
1603.297453388365
1605.3931058738535
1632.5404991800374
1607

4176.70149843498
4197.051079369992
4160.743210263052
4164.774262997571
4112.14476227368
Mean Fitness = 4166.966408500903


## Visualize Results using DataFrame

In [7]:
ga_evaluation_results_df =pd.DataFrame(ga_evaluation_results)
ga_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds,Mean CPU_time (seconds),Mean Fitness,Mean MSE,Mean RMSE,Mean PSNR,Mean SSIM,Mean UQI,Regions,CPU_time (seconds),Fitness,MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,mMasi_entropy,max,"[118, 175]",0.748280,1931.620833,2298.847661,47.903832,14.531163,0.774509,0.937611,"[[118, 255, 255, 255, 255, 255, 255, 255, 255,...","[0.5958893299102783, 1.1465504169464111, 0.583...","[1938.398305333441, 1921.2857090901718, 1946.8...","[2315.3358993530273, 2451.7205505371094, 2376....","[48.11793739711863, 49.51485181778402, 48.7521...","[14.484663552965714, 14.236093935027718, 14.37...","[0.7911818785663393, 0.7792262557912678, 0.785...","[0.9389059733768387, 0.933484766947488, 0.9363..."
1,AirplaneF16.tiff,3,mMasi_entropy,max,"[94, 144, 187]",1.024333,2003.155727,1808.563450,42.314498,15.647126,0.767552,0.951804,"[[94, 255, 255, 187, 187, 187, 255, 255, 255, ...","[0.9237568378448486, 0.773874044418335, 0.5779...","[2012.6139386688455, 1999.9556337972517, 2009....","[2021.274570465088, 1854.667335510254, 1898.13...","[44.95858728279936, 43.06584883071799, 43.5675...","[15.0745504866957, 15.448143376491094, 15.3475...","[0.7887831901278607, 0.7593064139025851, 0.793...","[0.9465443943366063, 0.9493843199918108, 0.949..."
2,AirplaneF16.tiff,4,mMasi_entropy,max,"[75, 118, 161, 194]",0.876266,2043.644545,1513.426691,38.494026,16.524698,0.766569,0.960110,"[[75, 255, 194, 194, 194, 194, 194, 194, 194, ...","[0.9011399745941162, 0.8971543312072754, 1.057...","[2045.5365548812774, 2055.9983749322673, 2022....","[1885.5896072387695, 1280.143798828125, 1946.0...","[43.42337627636489, 35.7790972332747, 44.11395...","[15.376331850360366, 17.0582160408734, 15.2392...","[0.7817258553550651, 0.7212173470876521, 0.811...","[0.9497034085790317, 0.9655694658764731, 0.951..."
3,AirplaneF16.tiff,5,mMasi_entropy,max,"[70, 106, 142, 166, 198]",0.876769,2063.197117,1295.557463,35.409009,17.306491,0.766616,0.966246,"[[70, 255, 198, 198, 198, 198, 198, 198, 198, ...","[1.6472461223602295, 0.8233053684234619, 2.488...","[2061.1522568728415, 2073.2267782265953, 2052....","[1818.2325134277344, 1616.5942344665527, 1556....","[42.640737721429424, 40.206892872572894, 39.45...","[15.534309413649092, 16.044793353004827, 16.20...","[0.8203447017476955, 0.7760329918440272, 0.776...","[0.9576749218512319, 0.9606266849528379, 0.963..."
4,Lena.png,2,mMasi_entropy,max,"[100, 158]",0.743893,1931.822164,2710.441726,51.451930,13.998826,0.614256,0.878746,"[[255, 255, 255, 255, 255, 158, 255, 255, 255,...","[0.7614245414733887, 0.6440474987030029, 1.623...","[1954.4404553996014, 1950.9758535428543, 1913....","[2367.2250747680664, 1952.8702278137207, 2176....","[48.654137283154725, 44.19129131190579, 46.648...","[14.388408084383268, 15.224069763701022, 14.75...","[0.6190248903185123, 0.6307561679906727, 0.615...","[0.8907325430481768, 0.8916122561156851, 0.879..."
5,Lena.png,3,mMasi_entropy,max,"[80, 128, 176]",0.604451,2082.518774,1503.949890,38.495664,16.480858,0.682544,0.923894,"[[176, 176, 176, 176, 176, 176, 176, 176, 176,...","[0.442948579788208, 0.682661771774292, 0.58462...","[2050.409000505437, 2085.133618619585, 2068.84...","[1521.946102142334, 1256.5047035217285, 2245.1...","[39.01212762901216, 35.4472100950375, 47.38262...","[16.30681088168337, 17.139162422252618, 14.618...","[0.6536973309287344, 0.6809744120901272, 0.652...","[0.925542738297894, 0.9168478460227789, 0.9201..."
6,Lena.png,4,mMasi_entropy,max,"[73, 109, 142, 180]",0.576999,2143.223245,1096.007452,32.901965,17.838923,0.714120,0.941893,"[[180, 180, 180, 180, 180, 180, 180, 180, 180,...","[0.763106107711792, 0.5001678466796875, 0.4028...","[2144.600522414831, 2140.627495763507, 2095.08...","[1124.1721115112305, 1255.666088104248, 994.64...","[33.52867595821867, 35.43537904558448, 31.5379...","[17.622475537653013, 17.142061954903767, 18.15...","[0.7207143509156608, 0.7177975268242448, 0.697...","[0.9352421994152766, 0.9499544293033138, 0.934..."
7,L

# Save results

In [8]:
pickle.dump(ga_evaluation_results_df, open('results/evaluation/GA/GA_masi_df.pkl', 'wb'))
pickle.dump(ga_evaluation_results, open('results/evaluation/GA/GA_masi_dict.pkl', 'wb'))